<a href="https://colab.research.google.com/github/HaeunJeon/Jeju_Rec/blob/main/%ED%81%B4%EB%9F%AC%EC%8A%A4%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **파일불러오기**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
def bring():
  df = pd.read_excel("/content/drive/MyDrive/종합설계/final_review2.xlsx")
  df.to_csv
  df=df.loc[:,['place', 'review']]
  return df

# **군집화**

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
def clustering(content):
  # 군집화 할 그룹의 갯수 정의
  n_clusters = len(content)//50

    # CountVectrizer로 토큰화
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(content)

    # l2 정규화
  X = normalize(X)

    # k-means 알고리즘 적용
  kmeans = KMeans(n_clusters=n_clusters).fit(X)

    # trained labels and cluster centers
  labels = kmeans.labels_
  centers = kmeans.cluster_centers_

    # labels에 merge
  place['labels'] = labels
  df = place
  return kmeans

# **빈도수**

In [5]:
#빈도수 계산 위한 tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

# ============================================
# -- Get TFIDF
# ============================================
def tfidf(content):
  vectorizer = TfidfVectorizer()
  sp_matrix = vectorizer.fit_transform(content)

  word2id = defaultdict(lambda : 0)
  for idx, feature in enumerate(vectorizer.get_feature_names()):
      word2id[feature] = idx

  for i, sent in enumerate(content):
      print('====== document[%d] ======' % i)
      print( [ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ] )
  return vectorizer

# **상위 feature 출력**

In [6]:
def get_cluster_details(cluster_model, cluster_data, feature_names,
                       cluster_num, top_n_features=10):
    cluster_details = {}
    # 각 클러스터 레이블별 feature들의 center값들 내림차순으로 정렬 후의 인덱스를 반환
    center_feature_idx = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    # 개별 클러스터 레이블별로 
    for cluster_num in range(cluster_num):
        # 개별 클러스터별 정보를 담을 empty dict할당
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # 각 feature별 center값들 정렬한 인덱스 중 상위 20개만 추출
        top_ftr_idx = center_feature_idx[cluster_num, :top_n_features]
        top_ftr = [feature_names[idx] for idx in top_ftr_idx]
        # top_ftr_idx를 활용해서 상위 20개 feature들의 center값들 반환
        # 반환하게 되면 array이기 떄문에 리스트로바꾸기
        top_ftr_val = cluster_model.cluster_centers_[cluster_num, top_ftr_idx].tolist()
        
        # cluster_details 딕셔너리에다가 개별 군집 정보 넣어주기
        cluster_details[cluster_num]['top_features'] = top_ftr
        cluster_details[cluster_num]['top_featrues_value'] = top_ftr_val
        
    return cluster_details

def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print()
        print(f"#####Cluster Num: {cluster_num}")
        print()
        print("상위 10개 feature단어들:\n", cluster_detail['top_features'])
        print()
        print('-'*20)

In [11]:
place

,place,review,labels
25194,용두암,야시장 갔다가 LED 정원 보고 용두암 들러 용연다리 보고 코스가 좋은 듯,1
25195,용두암,제주 가을 핑크 뮬리 푸른 바다 용두암 비 오는 제주,1
25196,용두암,용두암에 들려 이쁜 사진 찍고 국립제주박물관에 들려 제주도에 대한 멋진 이야기 함께...,3
25197,용두암,제주 가을용두암,0
25198,용두암,24년 전 신혼여행으로 제주도에 가서 용두암 구경하고 바닷가에서 회 먹고 바닷가 배...,1
...,...,...,...
25559,용두암,이곳은 좋은 바다 전망 하지만 괜찮아요 한 제가 드래건 머리 암석을 알려주었습니다 ...,6
25560,용두암,놀라운 바다 전망 식사를 해도 해산물이 블록 ajummas 지원하지만 적은 최소 중...,6
25561,용두암,드래건 머리 바위는 흥미로운 사진 기념품 이 멋진 곳 하지만 아마 않습니다 중 하나...,2
25562,용두암,그냥 여기 와서 되는 상품을 너도 판 저 드래건 물기 아무것도 잃을 경우 여기에 방...,2


# **장소입력 및 실행**

In [16]:
#장소입력
where = '용두암'
df = bring()
place = df.loc[df.place == where]
content= place['review'].apply(lambda x : str(x))
kmeans = clustering(content)
vectorizer = tfidf(content)
feature_names = vectorizer.get_feature_names()
cluster_details = get_cluster_details(cluster_model=kmeans,
                                    cluster_data=content,
                                    feature_names=feature_names,
                                    cluster_num=len(content)//50,
                                    top_n_features=20)
print_cluster_details(cluster_details)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


====== document[0] ======
[('야시장', 0.33469530582926776), ('갔다가', 0.2974156880242781), ('LED', 0.0), ('정원', 0.33469530582926776), ('보고', 0.4391917422406952), ('용두암', 0.13905457385980105), ('들러', 0.33469530582926776), ('용연다리', 0.33469530582926776), ('보고', 0.4391917422406952), ('코스가', 0.33469530582926776), ('좋은', 0.1649019741798586), ('듯', 0.0)]
====== document[1] ======
[('제주', 0.44133947209404656), ('가을', 0.3787097377436047), ('핑크', 0.4051044459857858), ('뮬리', 0.4051044459857858), ('푸른', 0.4051044459857858), ('바다', 0.24068617834128114), ('용두암', 0.16830718902881683), ('비', 0.0), ('오는', 0.28846563419973265), ('제주', 0.44133947209404656)]
====== document[2] ======
[('용두암에', 0.22163299655424593), ('들려', 0.5188306347986856), ('이쁜', 0.24658712926041787), ('사진', 0.1461985492491157), ('찍고', 0.1876479939694694), ('국립제주박물관에', 0.2774956330973357), ('들려', 0.5188306347986856), ('제주도에', 0.21567862542350003), ('대한', 0.2594153173993428), ('멋진', 0.19072449271732805), ('이야기', 0.2774956330973357), ('함께하며',